<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 7. Conceptos avanzados</font></h1>

<h1><font color="#113D68" size=4>3. Herramientas para trabajar con redes neuronales</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [1. Carga y almacenamiento de modelos y pesos](#section1)
    * [1.1. Almacenamiento del modelo](#section11)
    * [1.2. Carga del modelo](#section12)
    * [1.3. Almacenamiento de los pesos](#section13)
    * [1.4. Carga de los pesos](#section14)
* [2. Callbacks y Checkpoints](#section2)
    * [2.1. Early Stopping](#section21)
    * [2.2. Checkpoint](#section22)
* [3. TensorBoard](#section3)
    * [3.1. Instalación](#section31)
    * [3.2. Cargamos Tensorboard](#section32)

In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import keras

%matplotlib inline

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Establecemos una semilla para numpy y tensorflow para poder reproducir la ejecución y los resultados
SEED = 1000
np.random.seed(SEED)
tf.random.set_seed(SEED)

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Carga y almacenamiento de modelos y pesos</font>

Es muy importante ser capaz de almacenar aquellos modelos aprendidos y poder desplegarlos en otros equipos. Una vez hemos entrenado una red neuronal, deberemos ser capaces de poder almacenarla y cargarla para poder utilizarla, por ejemplo, en producción o en dispositivos incapaces de llevar a cabo el entrenamiento, pero si la evaluación. Otro uso puede ser el llevar a cabo un proceso posterior de *finetuning* conforme aumenten el número de imágenes disponibles.

Vamos a ver las diferentes formas que tiene Keras de guardar y cargar los modelos. Lo primero es que Keras separa los pesos de la estructura del modelo, por eso primero veremos como almacenar la estructura de una red y luego los pesos aprendidos.

In [2]:
# Buscamos guardar nuestro modelo (Red neuronal entrenada) y poder cargar esos pesos y sesgos ya entrenados, para no volver a entrenar
# Creamos una red basica

from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(784,))) # Primera capa oculta de 32 neuronas
model.add(Dense(32, activation='relu')) # Segunda capa oculta de 32 neuronas
model.add(Dense(10, activation='softmax')) # Salida (10 clases, números de 0 a 9 a predecir)

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<a id="section11"></a>
# <font color="#004D7F" size=5>1.1. Almacenamiento del modelo</font>

En este caso es guardar únicamente la estructura del modelo, pero no guarda los esos (que lo veremos posteriormente)

In [3]:
# diccionario con la info (guardamos solo la estructura de la red, sin sus pesos)
# Cada capa está entre {}
model.get_config()

{'name': 'sequential',
 'trainable': True,
 'dtype': 'float32',
 'layers': [{'module': 'keras.layers',
   'class_name': 'InputLayer',
   'config': {'batch_shape': (None, 784),
    'dtype': 'float32',
    'sparse': False,
    'name': 'input_layer'},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'units': 32,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'registered_name': None,
   'build_config': {'input_shape': (None, 784)}},
  {'module': 'keras.layers',
  

In [5]:
# Podemos obtener también esas propiedades de la red en formato json
model.to_json()

'{"module": "keras", "class_name": "Sequential", "config": {"name": "sequential", "trainable": true, "dtype": "float32", "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 784], "dtype": "float32", "sparse": false, "name": "input_layer"}, "registered_name": null}, {"module": "keras.layers", "class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "units": 32, "activation": "relu", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "registered_name": null, "build_config": {"input_shape": [null, 784]}}, {"module": "keras.layers", "class_name": "Dense", "config": {"name": "dense_1", "tra

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
**En la nueva versión de Keras se encuetra _deprecated_ YAML. Lo pongo comentado porque ya no funciona este bloque**

In [5]:
###############   NO EJECUTAR ESTE BLOQUE - YAML DEPRECATED #################
# string para el yaml
# model.to_yaml()

'backend: tensorflow\nclass_name: Sequential\nconfig:\n  layers:\n  - class_name: InputLayer\n    config:\n      batch_input_shape: !!python/tuple\n      - null\n      - 784\n      dtype: float32\n      name: dense_input\n      ragged: false\n      sparse: false\n  - class_name: Dense\n    config:\n      activation: relu\n      activity_regularizer: null\n      batch_input_shape: !!python/tuple\n      - null\n      - 784\n      bias_constraint: null\n      bias_initializer:\n        class_name: Zeros\n        config: {}\n      bias_regularizer: null\n      dtype: float32\n      kernel_constraint: null\n      kernel_initializer:\n        class_name: GlorotUniform\n        config:\n          seed: null\n      kernel_regularizer: null\n      name: dense\n      trainable: true\n      units: 32\n      use_bias: true\n  - class_name: Dense\n    config:\n      activation: relu\n      activity_regularizer: null\n      bias_constraint: null\n      bias_initializer:\n        class_name: Zeros\n 

Almacenamos el diccionario con NumPy

In [10]:
# Almacenamos cada elemento con Numpyy (el diccionario)
np.save('model_dict.npy',model.get_config()) # le ponemos el nombre al archivo y guardamos la configuración de la estructura de la red en formato npy

# Creamos el fichero para poder cargar y guardar la red en formato json y ymal
f= open('model_json.json','w+') # guardamos solo la estructura del modelo en formato json
f.write(model.to_json())
f.close()

###############   NO EJECUTAR ESTE BLOQUE - YAML DEPRECATED #################
#f= open("model_yaml.yaml","w+")
#f.write(model.to_yaml())
#f.close()

<a id="section12"></a>
# <font color="#004D7F" size=5>1.2. Carga del modelo</font>

Primero cargamos el diccionario con la estructura del modelo

In [7]:
# Cargamos el modelo en formato numpy
model_struct = np.load('model_dict.npy', allow_pickle=True).item()
model2 = Sequential.from_config(model_struct)
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │          25,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,506 (103.54 KB)

 Trainable params: 26,506 (103.54 KB)

 Non-trainable params: 0 (0.00 B)

Cargamos el modelo en JSON

In [11]:
# Cargamos el modelo en formato json
from keras.models import model_from_json

model2 = model_from_json(open('model_json.json','rb').read()) # lo ponemos en modo lectura binaria
model2.summary()

# Vemos que los datos todavía estan sin entrenar 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │          25,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,506 (103.54 KB)

 Trainable params: 26,506 (103.54 KB)

 Non-trainable params: 0 (0.00 B)

Cargamos el modelo en YAML

In [9]:
###############   NO EJECUTAR ESTE BLOQUE - YAML DEPRECATED #################
#from keras.models import model_from_yaml

#model2 = model_from_yaml(open("model_json.json","rb").read())
#model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                25120     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 26,506
Trainable params: 26,506
Non-trainable params: 0
_________________________________________________________________


<a id="section13"></a>
# <font color="#004D7F" size=5>1.3. Almacenamiento de los pesos</font>

Ya podríamos almacenar los pesos, pero como estos son por defecto, vamos a entrenar rápidamente una red para el problema de MNIST.

In [6]:
# Si queremos almacenar el modelo en conjunto con los pesos correspondientes luego de entrenarlo, hacemos lo siguiente

# Primero entrenamos rápidamente a la red
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

# cargamos la DB
(x_train_valid, y_train_valid), (x_test, y_test) = mnist.load_data()

# dividimos en train/valid, estando test ya identificada
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_valid, y_train_valid, test_size=0.1, random_state=SEED, stratify=y_train_valid)

# preprocesamos los datos
x_train = x_train.astype('float32')
x_train /= 255
x_train = x_train.reshape(x_train.shape[0],-1)
y_train = keras.utils.to_categorical(y_train, num_classes=10)
print("Dimensiones del conjunto de características de train aplanadas: {}".format(x_train.shape))
print("Dimensiones del conjunto de etiquetas de train en one hot: {}".format(y_train.shape))
print()

x_valid = x_valid.astype('float32')
x_valid /= 255
x_valid = x_valid.reshape(x_valid.shape[0],-1)
y_valid = keras.utils.to_categorical(y_valid, num_classes=10)
print("Dimensiones del conjunto de características de train aplanadas: {}".format(x_valid.shape))
print("Dimensiones del conjunto de etiquetas de train en one hot: {}".format(y_valid.shape))
print()


x_test = x_test.astype('float32')
x_test /= 255
x_test = x_test.reshape(x_test.shape[0],-1)
y_test = keras.utils.to_categorical(y_test, num_classes=10)
print("Dimensiones del conjunto de características de test aplanadas: {}".format(x_test.shape))
print("Dimensiones del conjunto de etiquetas de test en one hot: {}".format(y_test.shape))

# Tenemos 54000 imágenes para entrenamiento, 6000 para validación y 10000 para testeo

Dimensiones del conjunto de características de train aplanadas: (54000, 784)
Dimensiones del conjunto de etiquetas de train en one hot: (54000, 10)

Dimensiones del conjunto de características de train aplanadas: (6000, 784)
Dimensiones del conjunto de etiquetas de train en one hot: (6000, 10)

Dimensiones del conjunto de características de test aplanadas: (10000, 784)
Dimensiones del conjunto de etiquetas de test en one hot: (10000, 10)


Compilamos el modelo con nuestro dataset anterior

In [13]:
# Entrenamos el modelo
model.compile(loss='categorical_crossentropy', optimizer='sgd')

history = model.fit(x_train, y_train, validation_data=(x_valid,y_valid), epochs=10, batch_size= 128) # ponemos 10 épocas asi se entrena rápido

# Una vez entrenado ya tenemos los pesos y umbrales(sesgo) asignados por la compilación. El umbral es una forma de compensar la función de activación de la neurona o fijar un nivel mínimo de actividad de la neurona para considerarse como activa (regulariza el aprendizaje)

Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.0252 - val_loss: 0.9456
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7794 - val_loss: 0.5377
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4931 - val_loss: 0.4388
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4100 - val_loss: 0.3932
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3689 - val_loss: 0.3664
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3435 - val_loss: 0.3473
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3250 - val_loss: 0.3323
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3105 - val_loss: 0.3200
Epoch 9/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2985 - val_loss: 0.3094
Epoch 10/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2881 - val_loss: 0.3001


Una vez compilado podemos obtener los pesos y los umbrales asignados de la compilación. Recordemos como es la unidad fundamental de una neurona donde podemos ver que además de los pesos, el proceso de compilación, asigna también unos valores relativos a los umbrales. En concreto los umbrales se definen como:
 + El **umbral,** el cual representa el grado de inhibición de la neurona, es un término constante que no depende del valor que tome la entrada. El umbral puede entenderse como una manera de compensar la función de activación, o una forma de fijar un nivel mínimo de actividad a la neurona para considerarse como activa. La suma ponderada de las entradas debe producir un valor mayor que $u$ para cambiar la neurona de estado 0 a 1. 
 
<img src="img/neurona.jpg">


Por tanto, como tenemos 3 capas en nuestro modelo tenemos 6 arrays de parámetros, los cuales, los dos primeros arrays corresponde al valor de los pesos (el primero) y los umbrales (el segundo), los dos siguientes a la segunda capa y los últimos a la última capa.

In [14]:
# Podemos ver el valor de los parámetros asignados (al entrenamiento específico realizado)
model.get_weights()

# Nos muestra 6 arrays. Los 2 primeros pertenecen a la primera capa oculta, el primero a los pesos(32 arreglos, uno para los pesos de cada naurona) y el segundo a los umbrales(32 valores);
# los 2 siguientes a los pesos y umbrales de la segunda capa oculta; y los últimos 2 a los pesos y umbrales de la capa de salida (en este caso posee 10 neuronas por lo que cada arreglo tiene 10 valores y el umbral solo tiene 10 valores)


[array([[-0.03862364, -0.00397223, -0.05359059, ...,  0.06350475,
          0.07604041,  0.05544551],
        [-0.07427593,  0.05336181, -0.07550387, ...,  0.0227139 ,
         -0.03288995,  0.04950372],
        [ 0.04723411,  0.01795948,  0.07037921, ..., -0.07459584,
          0.01123744, -0.07596777],
        ...,
        [ 0.07359526,  0.02797133, -0.06526592, ...,  0.00014466,
         -0.04730898,  0.05025419],
        [-0.02346993, -0.05873499, -0.01016098, ..., -0.02440092,
         -0.07039852, -0.07671025],
        [-0.03438187, -0.05634336, -0.06539854, ...,  0.04743197,
         -0.00710706, -0.03531517]], dtype=float32),
 array([ 0.05142695, -0.00070586,  0.01783496,  0.06568229,  0.08002208,
        -0.07578573,  0.00793004,  0.07037205,  0.14684124,  0.02179336,
        -0.08021172,  0.05829277,  0.04839266,  0.01033035,  0.00087553,
         0.02552476,  0.01753437,  0.14581184,  0.02108272,  0.09092388,
         0.00257584,  0.09685087,  0.02400576, -0.01538042,  0.144

Podemos almacenar con un array NumPy

In [16]:
###############   NO EJECUTAR ESTE BLOQUE - npy DEPRECATED #################
# np.save('weights_numpy.npy',model.get_weights())

La otra forma es la de formato tanto estructura como en valores con H5

In [17]:
# Este formato nos guarda la estructura del modelo y los pesos entrenados conjuntamente
model.save('model_h5.h5')

<a id="section14"></a>
# <font color="#004D7F" size=5>1.4. Carga de los pesos</font>

Ahora podemos cargar los pesos. Si primero lo cargamos con Numpy primero cargamos los pesos y luego tenemos que establecerlos a model2 con la función `set_weights`.

In [19]:
###############   NO EJECUTAR ESTE BLOQUE - npy DEPRECATED #################
#model2.set_weights(np.load('weights_numpy.npy', allow_pickle=True))

In [1]:
###############   NO EJECUTAR ESTE BLOQUE - npy DEPRECATED #################
#model2.get_weights()

Lo cargamos ahora con la forma H5 donde carga en un solo paso la estructura y los valores.

In [18]:
# Cargamos la estructura y los pesos
from keras.models import load_model

model3 = load_model('model_h5.h5')

In [19]:
# vemos la estructura guardada
model3.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │          25,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,508 (103.55 KB)

 Trainable params: 26,506 (103.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [21]:
# vemos los pesos guardados
model3.get_weights()

[array([[-0.03862364, -0.00397223, -0.05359059, ...,  0.06350475,
          0.07604041,  0.05544551],
        [-0.07427593,  0.05336181, -0.07550387, ...,  0.0227139 ,
         -0.03288995,  0.04950372],
        [ 0.04723411,  0.01795948,  0.07037921, ..., -0.07459584,
          0.01123744, -0.07596777],
        ...,
        [ 0.07359526,  0.02797133, -0.06526592, ...,  0.00014466,
         -0.04730898,  0.05025419],
        [-0.02346993, -0.05873499, -0.01016098, ..., -0.02440092,
         -0.07039852, -0.07671025],
        [-0.03438187, -0.05634336, -0.06539854, ...,  0.04743197,
         -0.00710706, -0.03531517]], dtype=float32),
 array([ 0.05142695, -0.00070586,  0.01783496,  0.06568229,  0.08002208,
        -0.07578573,  0.00793004,  0.07037205,  0.14684124,  0.02179336,
        -0.08021172,  0.05829277,  0.04839266,  0.01033035,  0.00087553,
         0.02552476,  0.01753437,  0.14581184,  0.02108272,  0.09092388,
         0.00257584,  0.09685087,  0.02400576, -0.01538042,  0.144

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6> 2. Callbacks y Checkpoints</font>

Los `Callbacks` son funciones que se aplican durante el entrenamiento de una red neuronal. Estas sirven para almacenar datos como la loss en capa epoch o aplicar el `EarlyStopping`. 

Entre ellas, también están los `Checkpoints`, que permiten almacenar el estado del entrenamiento y de la red dependiendo de una serie de parámetros. 
+ Por poner un ejemplo, podemos almacenar la mejor red en base a un conjunto de validación o por si hay algún error en el sistema no perderemos todo el trabajo.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Vamos a ver como realizar todo esto con Keras. Existen diferentes funciones que se pueden encontrar en [Callbacks](https://keras.io/callbacks/)

<a id="section21"></a>
# <font color="#004D7F" size=5> 2.1. Early Stopping</font>

Primero veamos como podemos detener el entrenamiento en cuanto en la época siguiente no mejora la métrica respecto a la anterior

In [22]:
# Primero el EarlyStopping. Detiene el entrenamiento cuando la época siguiente no mejora la métrica respecto a la anterior (cuando empeora, se para)
from keras.callbacks import EarlyStopping # ver más callbacks en documentación

early_stop = EarlyStopping(monitor='val_loss') # le decimos que monitoree y detenga la ejecución en función de la pérdida (podemos ponerlo tmb segun'loss')

In [24]:
history = model.fit(x_train, y_train, validation_data=(x_valid,y_valid), epochs=100, batch_size= 128,callbacks=[early_stop]) # ponemos la llamada que creamos (ver más llamadas en documentación)
# En este caso, en ninguna época se aumenta la pérdida con respecto a la época anterior, por lo que no se para el entrenamiento antes de las 50 épocas determinadas de base (50)
# Pero vemos que si aumentamos las épocas bases a 100, se corta en la época 76 el entrenamiento, ya que el val_loss se estanca

Epoch 1/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1197 - val_loss: 0.1606
Epoch 2/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1184 - val_loss: 0.1598
Epoch 3/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1172 - val_loss: 0.1591
Epoch 4/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1160 - val_loss: 0.1584
Epoch 5/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1148 - val_loss: 0.1577
Epoch 6/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1136 - val_loss: 0.1570
Epoch 7/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1124 - val_loss: 0.1563
Epoch 8/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1113 - val_loss: 0.1557
Epoch 9/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1101 - val_loss: 0.1551
Epoch 10/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1091 - val_loss: 0.1545
Epoch 11/100
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1080 - val_loss: 0.1539
Epoch 12/100
422/422 ━━━━━━━━━━━━━━━━━━━━

<a id="section22"></a>
# <font color="#004D7F" size=5> 2.2. Checkpoint</font>

Veamos ahora como funciona Checkpoint. Recordemos que `ModelCheckpoint` nos permitía almacenar distintos estados del modelo conforme vaya avanzando el entrenamiento del modelo.

In [26]:
# Y ahora los Checkpoints. Permite almacenar los distintos estados del modelo conforme se va entrenando
# Podemos decirle que vaya guardando el modelo conforme las métricas del mismo vayan mejorando
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(filepath='model.checkpoint/mnist_model_{epoch:02d}.keras') # le pedimos que me guarde el modelo con el número de época en formato keras(transformar de keras a h5) en la carpeta de checkpoint

In [27]:
history = model.fit(x_train, y_train, validation_data=(x_valid,y_valid), epochs=10, batch_size= 128,callbacks=[checkpoint])
# Si vemos en la carpeta nos guarda el modelo entrenado para cada época
# Se usa para hacer experimentos controlados con nuestra red

Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0625 - val_loss: 0.1406
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0620 - val_loss: 0.1405
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0616 - val_loss: 0.1405
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0611 - val_loss: 0.1405
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0607 - val_loss: 0.1405
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0603 - val_loss: 0.1405
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0598 - val_loss: 0.1405
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0594 - val_loss: 0.1405
Epoch 9/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0590 - val_loss: 0.1404
Epoch 10/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0586 - val_loss: 0.1405


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. TensorBoard</font>

`TensorBoard` es una herramienta muy potente de visualización desarrollada en conjunto con *TensorFlow*. Esta herramienta nos permite ver en tiempo real la evolución del entrenamiento de nuestras redes y nos permite detectar problemas o poder realizar optimizaciones.

Desde *Keras* es posible generar los ficheros necesarios para la visualización del entrenamiento en `TensorBoard`. Esto se hace mediante otro `Callback`. 

Sin embargo, `TensorBoard` se visualiza a través del navegador, por lo que al estar trabajando en *Colab* es necesario hacer un tunel a la máquina donde esta alojado nuestro cuaderno. No vamos a entrar en detalles con esto, ya que solo afecta a *Colab*, pero os dejamos los comandos que se pueden copiar a cualquier otro *notebook* para poder conectar con `TensorBoard`.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
La curva de aprendizaje de esta herramienta es amplia por lo que no vamos a entrar en muchos detalles. Para más información ver la documentación oficial de [`TensorBoard`](https://www.tensorflow.org/tensorboard)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Tutorial para instalar [`TensorBoard`](https://www.dlology.com/blog/how-to-run-tensorboard-in-jupyter-notebook/)

<a id="section31"></a>
# <font color="#004D7F" size=5>3.1. Instalación</font>

----
Para **Jupyter Notebook** podemos ejecutar en el mismo cuaderno o también en una pestaña nueva del navegador. 

Para verlo en una pestaña nueva hay que poner la dirección `http://localhost:6006` una vez cargado los *magic* como se indica a continuación.

In [1]:
# TensorBoard nos permite ver la evolución del entrenamiento de nuestras redes y detectar problemas y optimizaciones a realizar
# Se utiliza mediante callbacks en Jupyter (en Colab se debe hacer de una manera diferente). Ver más funcionalidades de TensorBoard en la documentación de TensorFlow
import os
#!pip install tensorboard
%load_ext tensorboard

LOG_DIR = './keras_log' # creamos una carpeta keras_log
os.makedirs(LOG_DIR, exist_ok=True) # y cargamos los datos almacenados en TensorBoard en esa carpeta, utilizando un callback luego
%tensorboard --logdir {LOG_DIR}

# Como la visualización de TensorBoard en la libreta Jupyter es complicada, copiamos http://localhost:6006 y pegamos en url de una nueva pestaña para verlo desde allí

<a id="section32"></a>
# <font color="#004D7F" size=5>3.2. Cargamos Tensorboard</font>

Una vez cargada nuestra herramienta volvemos a crear nuestro modelo y le realizamos un callback para poder verlo en `TensorBoard`

In [2]:
# Reiniciamos para ver que todo funciona correctamente (cargamos nuevamente nuestro modelo)

from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(784,))) # Primera capa oculta
model.add(Dense(32, activation='relu')) # Segunda capa oculta
model.add(Dense(10, activation='softmax')) # Salida (10 clases)


model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
from keras.callbacks import TensorBoard

board = TensorBoard(log_dir=LOG_DIR) # objeto que, además de usar TensorBoard, carga los archivos en la carpeta

In [10]:
# Realizamos el entrenamiento y realizamos un callback para poder ver el modelo en TensorBoard
tf.profiler.experimental.stop

history = model.fit(x_train, y_train, validation_data=(x_valid,y_valid), epochs=50, batch_size= 128,callbacks=[board])

# Si actualizamos el TensorBoard, me aparece el progreso en las épocas
# Smothing es el suavizado, la parte de Graphs es como se construyó el grafo de cada una de las capas (pudiendo incluso configurarlo)
# Podemos visualizar la evolución de la pérdida y el accuracy de los datos de entrenamiento y validación (se nos guardan en la carpeta)

Epoch 1/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - acc: 0.9622 - loss: 0.1283 - val_acc: 0.9542 - val_loss: 0.1641
Epoch 2/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9627 - loss: 0.1267 - val_acc: 0.9552 - val_loss: 0.1630
Epoch 3/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9631 - loss: 0.1251 - val_acc: 0.9552 - val_loss: 0.1619
Epoch 4/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9635 - loss: 0.1236 - val_acc: 0.9560 - val_loss: 0.1609
Epoch 5/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - acc: 0.9639 - loss: 0.1221 - val_acc: 0.9562 - val_loss: 0.1599
Epoch 6/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9642 - loss: 0.1207 - val_acc: 0.9567 - val_loss: 0.1590
Epoch 7/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9646 - loss: 0.1193 - val_acc: 0.9570 - val_loss: 0.1581
Epoch 8/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9650 - loss: 0.1179 - val_acc: 0.9573 - val_loss: 0.1572
Epoch 9/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - ac

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>